In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '1'
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' 
import tensorflow as tf
tf.get_logger().setLevel('ERROR')
import pandas as pd
tf.config.set_soft_device_placement(False)
tf.debugging.set_log_device_placement(True)
import numpy as np
from ampligraph.datasets import load_fb15k_237, load_yago3_10
from ampligraph.evaluation.protocol import create_mappings, to_idx

from ampligraph.evaluation.metrics import mrr_score, hits_at_n_score, mr_score
import time
print(tf.__version__)
assert(tf.__version__.startswith('2.3'))

from ampligraph.datasets import load_fb15k_237, load_fb13, load_fb15k, load_wn11, load_wn18, load_wn18rr, load_yago3_10
from ampligraph.latent_features import ScoringBasedEmbeddingModel

2.3.0


In [2]:
#loss function
def nll(scores_pos, scores_neg, eta):
    scores_neg_reshaped = tf.reshape(scores_neg, [eta, tf.shape(scores_pos)[0]])
    neg_exp = tf.exp(scores_neg_reshaped)
    pos_exp = tf.exp(scores_pos)
    softmax_score = pos_exp / (tf.reduce_sum(neg_exp, axis=0) + pos_exp)

    loss = -tf.reduce_sum(tf.math.log(softmax_score))
    return loss

# Train/eval without partition

In [3]:
model = ScoringBasedEmbeddingModel(eta=50, 
                                     k=300, 
                                     max_ent_size=14505, 
                                     max_rel_size=237,
                                     scoring_type='TransE')

model.compile(optimizer='adam', loss=nll)

from tensorflow.keras.callbacks import ModelCheckpoint
checkpoint = ModelCheckpoint('./chkpt1', monitor='loss', verbose=1, save_best_only=True, mode='min')

model.fit('/home/spai/code/ampligraph_projects/dataset/fb15k-237/train.txt',
             batch_size=10000,
             epochs=10,
             validation_freq=5,
             validation_batch_size=100,
             validation_data = '/home/spai/code/ampligraph_projects/dataset/fb15k-237/valid.txt')




Epoch 1/10
29/29 [==============================] - 1s 47ms/step - loss: 37952.4688
Epoch 2/10
29/29 [==============================] - 1s 37ms/step - loss: 37624.0742
Epoch 3/10
29/29 [==============================] - 1s 37ms/step - loss: 37223.4375
Epoch 4/10
29/29 [==============================] - 1s 37ms/step - loss: 36722.4180
Epoch 5/10
28/29 [===========================>..] - ETA: 0s - loss: 36336.5195
9 triples containing invalid keys skipped!
29/29 [==============================] - 6s 216ms/step - loss: 36128.2305 - val_mrr: 0.0959 - val_mr: 1045.5039 - val_hits@1: 0.0000e+00 - val_hits@10: 0.2593
Epoch 6/10
29/29 [==============================] - 1s 37ms/step - loss: 35482.7773
Epoch 7/10
29/29 [==============================] - 1s 37ms/step - loss: 34822.5273
Epoch 8/10
29/29 [==============================] - 1s 37ms/step - loss: 34163.2930
Epoch 9/10
29/29 [==============================] - 1s 37ms/step - loss: 33513.8438
Epoch 10/10
28/29 [===========================>

In [ ]:

start = time.time()
ranks = model.evaluate('/home/spai/code/ampligraph_projects/dataset/fb15k-237/test.txt', 
                       batch_size=100,
                       corrupt_side='s,o',
         use_filter={'train':'/home/spai/code/ampligraph_projects/dataset/fb15k-237/train.txt',
                  'valid':'/home/spai/code/ampligraph_projects/dataset/fb15k-237/valid.txt',
                  'test':'/home/spai/code/ampligraph_projects/dataset/fb15k-237/test.txt'})
end = time.time()
print('Time taken:', end-start)
mr_score(ranks), mrr_score(ranks), hits_at_n_score(ranks, 1), hits_at_n_score(ranks, 10)


28 triples containing invalid keys skipped!

9 triples containing invalid keys skipped!

28 triples containing invalid keys skipped!
      6/Unknown - 1s 219ms/step

# Training/eval with partition

In [5]:
from ampligraph.datasets import DummyBackend, SQLiteAdapter
from ampligraph.datasets import GraphDataLoader
from ampligraph.datasets.graph_partitioner import PARTITION_ALGO_REGISTRY
dataset_loader = GraphDataLoader('/home/spai/code/ampligraph_projects/dataset/fb15k-237/train.txt', 
                                 backend=SQLiteAdapter,
                                    batch_size=1000, 
                                    dataset_type='train', 
                                    epochs=10,
                                    use_indexer=True)


partitioner = PARTITION_ALGO_REGISTRY.get('Bucket')(dataset_loader, k=3)

_split: memory before: 896.0Bytes, after: 13.121MB, consumed: 13.121MB; exec time: 210.47s


In [6]:
partitioned_model = ScoringBasedEmbeddingModel(eta=5, 
                                     k=300, 
                                     max_ent_size=9437, 
                                     max_rel_size=237,
                                     scoring_type='TransE')

partitioned_model.compile(optimizer='adam', loss=nll)

from tensorflow.keras.callbacks import ModelCheckpoint
checkpoint = ModelCheckpoint('./chkpt1', monitor='loss', verbose=1, save_best_only=True, mode='min')

partitioned_model.fit(partitioner,
         batch_size=1000, use_partitioning=True,
         epochs=10, callbacks=[checkpoint])

Epoch 1/10
    277/Unknown - 11s 38ms/step - loss: 1675.2634
Epoch 00001: loss improved from inf to 1675.26343, saving model to ./chkpt1
277/277 [==============================] - 11s 41ms/step - loss: 1675.2634
Epoch 2/10
276/277 [============================>.] - ETA: 0s - loss: 1553.0312
Epoch 00002: loss improved from 1675.26343 to 1550.38745, saving model to ./chkpt1
277/277 [==============================] - 9s 32ms/step - loss: 1550.3875
Epoch 3/10
276/277 [============================>.] - ETA: 0s - loss: 1435.3806
Epoch 00003: loss improved from 1550.38745 to 1433.75256, saving model to ./chkpt1
277/277 [==============================] - 9s 32ms/step - loss: 1433.7526
Epoch 4/10
276/277 [============================>.] - ETA: 0s - loss: 1332.5895
Epoch 00004: loss improved from 1433.75256 to 1331.45679, saving model to ./chkpt1
277/277 [==============================] - 9s 33ms/step - loss: 1331.4568
Epoch 5/10
276/277 [============================>.] - ETA: 0s - loss: 1244.68

In [7]:

dataset_loader_test = GraphDataLoader('/home/spai/code/ampligraph_projects/dataset/fb15k-237/test.txt', 
                                        backend=SQLiteAdapter,
                                        batch_size=400, 
                                        dataset_type='test', 
                                        epochs=1,
                                        use_indexer=partitioned_model.data_handler.get_mapper())

start = time.time()
ranks = partitioned_model.evaluate(dataset_loader_test, 
                                   batch_size=400)
end = time.time()
print('Time taken:', end-start)

mr_score(ranks), mrr_score(ranks), hits_at_n_score(ranks, 1), hits_at_n_score(ranks, 10)


28 triples containing invalid keys skipped!
52/52 [==============================] - 124s 2s/step
Time taken: 127.01995515823364


(1264.7075490196078, 0.08407721567857451, 0.0, 0.2399264705882353)

In [8]:
start = time.time()
ranks = partitioned_model.evaluate('/home/spai/code/ampligraph_projects/dataset/fb15k-237/test.txt', 
                       batch_size=200,
                       corrupt_side='s,o',
                        use_filter={'train':'/home/spai/code/ampligraph_projects/dataset/fb15k-237/train.txt',
                              'valid':'/home/spai/code/ampligraph_projects/dataset/fb15k-237/valid.txt',
                              'test':'/home/spai/code/ampligraph_projects/dataset/fb15k-237/test.txt'})
end = time.time()
print('Time taken:', end-start)
mr_score(ranks), mrr_score(ranks), hits_at_n_score(ranks, 1), hits_at_n_score(ranks, 10)


28 triples containing invalid keys skipped!

9 triples containing invalid keys skipped!

28 triples containing invalid keys skipped!
104/104 [==============================] - 709s 7s/step
Time taken: 767.4920349121094


(1142.7554555240238,
 0.18002508091586358,
 0.11167922497308934,
 0.31015754966239356)